# Topic modeling

## Comparación entre Propuesta de Constitución y Constitución Vigente

### Librerías

In [1]:
import pandas as pd
import numpy as np

In [2]:
# for text preprocessing
import re
import spacy

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [3]:
# import LDA from sklearn
from sklearn.decomposition import LatentDirichletAllocation

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
esp_stop = stopwords.words('spanish')
esp_stop = esp_stop + ['ser','tener','deber']

## Funciones

In [6]:
def delete_stopwords(text):
    """
    Función para eliminar stopwords del texto
    """
    doc = nlp(text)
    return ' '.join([word.text for word in doc if word.text not in esp_stop])

In [7]:
def cleaner(df, text):
    """
    Función para dejar las palabras en minúscula, eliminar números, puntuación, espacios y saltos de línea.
    """
    df[text] = df[text].str.lower()
    df[text] = df[text].apply(lambda x: re.sub("[0-9]", '', str(x)))
    df[text] = df[text].str.translate(str.maketrans(' ', ' ', string.punctuation))
    df[text] = df[text].str.replace('\n',' ')
    df[text] = df[text].str.replace('nº','')
    df[text] = df[text].str.replace('n°','')
    df[text] = df[text].str.replace('n º','')
    df[text] = df[text].str.replace('n °','')
    df[text] = df[text].str.replace('º','')
    df[text] = df[text].str.replace('°','')
    df[text] = df[text].str.strip()
    df[text] = df[text].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')  # Eliminación de acentos
    

In [8]:
def wordcloud(data):
    """
    Función para graficar un wordcloud
    """
    wc = WordCloud(background_color='white',width=800, height=400, max_words=200, colormap = 'plasma').generate_from_frequencies(data)
    plt.figure(figsize=(15, 10))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.show()

### Importando textos ya digitalizados

In [9]:
df_propuesta = pd.read_csv('data/processed/constitucion_propuesta.csv')
df_propuesta.rename(columns = {'articulo':'nro_articulo'}, inplace = True)
df_propuesta['constitucion'] = 'Propuesta'
df_propuesta = df_propuesta[df_propuesta['capitulo']!='Convención Constitucional']

df_vigente = pd.read_csv('data/processed/constitucion_actual.csv')
df_vigente.drop(columns = 'id', inplace = True)
df_vigente['constitucion'] = 'Vigente'

df = pd.concat([df_vigente,df_propuesta])

df = df[['constitucion','capitulo', 'titulo_capitulo', 'nro_articulo', 'texto']]

In [10]:
df.reset_index(drop = True, inplace = True)

In [11]:
df.sample(3)

,constitucion,capitulo,titulo_capitulo,nro_articulo,texto
1482,Propuesta,Capítulo VI,Estado Regional y Organización Territorial,Artículo 164,- i) El fomento y la protección a las cultura...
2018,Propuesta,Capítulo VIII,Poder Ejecutivo,Artículo 263,### Artículo 291
1361,Propuesta,Capítulo V,Buen Gobierno y Función Pública,Artículo 164,### Artículo 170


### Desarrollo

Basado en: https://github.com/sethns/Latent-Dirichlet-Allocation-LDA-/blob/main/Topic%20Modeling%20_%20Extracting%20Topics_%20Using%20Sklearn.ipynb

Definiendo nlp

In [12]:
nlp = spacy.load('es_core_news_sm')

Limpiando texto

In [13]:
cleaner(df, 'texto')

In [14]:
df['texto_clean'] = df['texto'].apply(lambda x: delete_stopwords(x))

In [15]:
df.sample()

,constitucion,capitulo,titulo_capitulo,nro_articulo,texto,texto_clean
955,Propuesta,Capítulo II,Derechos Fundamentales y Garantías,Artículo 40,toda persona tiene derecho a recibir una educa...,toda persona derecho recibir educacion sexual ...


In [16]:
cv_vectorizer = CountVectorizer(ngram_range = (1,1))
cv_arr = cv_vectorizer.fit_transform(df['texto_clean'])

In [17]:
cv_arr

<2619x5615 sparse matrix of type '<class 'numpy.int64'>'
	with 41120 stored elements in Compressed Sparse Row format>

In [18]:
# Creating vocabulary array which will represent all the corpus 
vocab_cv = cv_vectorizer.get_feature_names()

# get the vocb list
vocab_cv[:10]

['abandono',
 'abastecer',
 'abastecimiento',
 'abierta',
 'abiertamente',
 'abierto',
 'abiertos',
 'abogada',
 'abogadas',
 'abogado']

### Implementando LDA

In [19]:
# Implementation of LDA:
    
# Create object for the LDA class 
# Inside this class LDA: define the components:
lda_model = LatentDirichletAllocation(n_components = 7, max_iter = 20, random_state = 20)

# fit transform on model on our count_vectorizer : running this will return our topics 
X_topics = lda_model.fit_transform(cv_arr)

# .components_ gives us our topic distribution 
topic_words = lda_model.components_



In [20]:
#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words = 10

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_cv)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i), topic_words)

Topic 0 ['ley' 'constitucion' 'republica' 'atribuciones' 'nacional' 'podra'
 'presidente' 'organizacion' 'funciones']
Topic 1 ['derecho' 'ley' 'constitucion' 'persona' 'toda' 'podra' 'derechos'
 'ejercicio' 'organos']
Topic 2 ['articulo' 'ley' 'inciso' 'sera' 'tribunal' 'podra' 'leyes' 'perjuicio'
 'constitucion']
Topic 3 ['derechos' 'ley' 'derecho' 'personas' 'ejercicio' 'educacion' 'humanos'
 'constitucion' 'proteccion']
Topic 4 ['regional' 'territoriales' 'region' 'constitucion' 'entidades' 'gobierno'
 'autonoma' 'administracion' 'competencias']
Topic 5 ['ley' 'constitucional' 'organica' 'fuerza' 'texto' 'decreto' 'electoral'
 'articulo' 'general']
Topic 6 ['presidente' 'republica' 'diputados' 'congreso' 'anos' 'camara' 'corte'
 'cargo' 'si']


Creando df de tópicos

In [21]:
df_topicos = pd.DataFrame(X_topics)
df_topicos.columns = ['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3','Topic 4','Topic 5','Topic 6']

Agrando los tópicos al df original

In [22]:
df_con_topicos = pd.merge(df, df_topicos, left_index = True, right_index = True)
df_con_topicos['max_topic'] = X_topics.argmax(axis = 1)

### Cuanto aparece cada tópico en cada constitución?

In [30]:
temp = df_con_topicos[df_con_topicos['constitucion']=='Propuesta'].groupby(['titulo_capitulo','max_topic'])['texto'].count().reset_index()
temp['peso'] = temp['texto']/temp['texto'].sum()
temp[['titulo_capitulo','max_topic','peso']]

,titulo_capitulo,max_topic,peso
0,Buen Gobierno y Función Pública,0,0.006791
1,Buen Gobierno y Función Pública,1,0.001698
2,Buen Gobierno y Función Pública,2,0.013016
3,Buen Gobierno y Función Pública,3,0.013016
4,Buen Gobierno y Función Pública,4,0.004527
...,...,...,...
76,Órganos Autónomos Constitucionales,2,0.021505
77,Órganos Autónomos Constitucionales,3,0.019242
78,Órganos Autónomos Constitucionales,4,0.004527
79,Órganos Autónomos Constitucionales,5,0.002830


In [31]:
temp = df_con_topicos[df_con_topicos['constitucion']!='Propuesta'].groupby(['titulo_capitulo','max_topic'])['texto'].count().reset_index()
temp['peso'] = temp['texto']/temp['texto'].sum()
temp[['titulo_capitulo','max_topic','peso']]

,titulo_capitulo,max_topic,peso
0,BANCO CENTRAL,0,0.002347
1,BANCO CENTRAL,3,0.002347
2,BANCO CENTRAL,4,0.001174
3,BANCO CENTRAL,5,0.001174
4,BASES DE LA INSTITUCIONALIDAD,0,0.003521
...,...,...,...
87,TRIBUNAL CONSTITUCIONAL,1,0.004695
88,TRIBUNAL CONSTITUCIONAL,2,0.010563
89,TRIBUNAL CONSTITUCIONAL,4,0.001174
90,TRIBUNAL CONSTITUCIONAL,5,0.002347
